In [1]:
import pandas as pd

## 1.Load the patient dataset and show summary with info().


In [2]:
billing=pd.read_csv("Billing_Data.csv")

In [3]:
patient=pd.read_csv("Patient_Data.csv")

In [4]:
billing.head()

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [5]:
patient.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [6]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [7]:
patient.duplicated(subset='PatientID')	

0    False
1    False
2    False
3    False
4    False
5     True
dtype: bool

In [8]:
billing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [9]:
patient.isnull().sum()

PatientID         0
Name              0
Department        0
Doctor            0
BillAmount        2
ReceptionistID    0
CheckInTime       0
dtype: int64

## 6.	Fill missing BillAmount values with the mean bill amount.

In [10]:
patient['BillAmount'] = patient['BillAmount'].fillna(patient['BillAmount'].mean())

In [11]:
billing.isnull().sum()

PatientID           0
InsuranceCovered    0
FinalAmount         0
dtype: int64

## 2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [12]:
billing_data=patient[['PatientID', 'Department', 'Doctor', 'BillAmount']]
billing_data

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,5925.0
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,5925.0
5,101,Cardiology,Dr. Smith,5000.0


## 3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [13]:
patient_admin_drop=patient.drop(['ReceptionistID', 'CheckInTime'],axis=1, errors='ignore', inplace=True)

## 4.	Use groupby to find total bill amount per department.

In [14]:
total_bill=billing_data.groupby('Department')['BillAmount'].sum().reset_index()
total_bill=billing_data.rename(columns={'BillAmount':'TotalBillAmount'})
total_bill

,PatientID,Department,Doctor,TotalBillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,5925.0
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,5925.0
5,101,Cardiology,Dr. Smith,5000.0


## 5.	Remove duplicate patient records based on PatientID.

In [15]:
patient.drop_duplicates(subset='PatientID',inplace=True)	

In [16]:
patient

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,5925.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,5925.0


## 7.	Merge the billing dataset with patient dataset on PatientID.

In [17]:
merged = pd.merge(patient,billing,on='PatientID',how='left')
merged

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000


## 8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [18]:
new_patients = pd.DataFrame({
    'PatientID': [201, 202],
    'Name': ['Kiran', 'Manoj'], 
    'Department': ['Cardiology', 'Orthopedics'],
    'Doctor': ['Dr.Sujal', 'Dr.Mallikarjun'],
    'BillAmount': [16000, 12000],
    'InsuranceCovered': [2500, 1000],
    'FinalAmount': [3300, 3300]
})

In [19]:
merged_new = pd.concat([merged, new_patients], axis=0,ignore_index=True)

In [20]:
merged_new

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000
5,201,Kiran,Cardiology,Dr.Sujal,16000.0,2500,3300
6,202,Manoj,Orthopedics,Dr.Mallikarjun,12000.0,1000,3300
